### 1. Keras笑脸识别
本部分的任务是读取图片并识别图片上的**笑脸**。
Keras是一个比Tensorflow层次更高的框架，一般来说通用的模型都可以通过Keras实现，但是一些复杂的模型在keras中没有，只能用较低层次的Tensorflow或pytorch实现
#### 1.1 引入库 and 加载数据集

In [10]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import kt_utils 

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

# 加载数据
X_train_orig,Y_train_orig,X_test,Y_test,classes = kt_utils.load_dataset()
X_train = X_train_orig / 255
Y_train = Y_train_orig / 255

Y_train = Y_train_orig.T
Y_test = Y_test.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


#### 1.2 使用keras框架构建训练模型
keras与tensorflow不一样的是，我们不需要创建如Z1,A1等的中间变量，只需要保存最后结果，所以在fp的时候只有一个tensor X。

In [12]:
def model(input_shape):
    """
    参数：
        input_shape - 输入参数的size
    返回：
        model - keras模型
    """
    # 类似于tensorflow中的创建占位符
    X_input = Input(input_shape)
    
    # zero padding
    # 由于后面我们要用到7*7的卷积 所以这里p=(7-1)/2=3
    X = ZeroPadding2D((3,3))(X_input)
    
    # Conv - BN - relu
    X = Conv2D(32,(7,7),strides=(1,1),name="conv0")(X)
    X = BatchNormalization(axis=3,name="bn0")(X)
    X = Activation('relu')(X)
    
    # MaxPooling
    X = MaxPooling2D((2,2),name="max_pool")(X)
    
    # 向量化
    X = Flatten()(X)
    # Fully Connected
    X = Dense(1,activation="sigmoid",name="fc")(X)
    
    model = Model(inputs = X_input, outputs = X, name="HappyModel")
    
    return model

In [14]:
# 获取模型
HappyModel = model(X_train.shape[1:])

# 编译模型
HappyModel.compile("adam","binary_crossentropy",metrics=['accuracy'])

# 训练模型
HappyModel.fit(X_train,Y_train,epochs=60,batch_size=50)

# 评估模型
preds = HappyModel.evaluate(X_test,Y_test,batch_size=32,verbose=1,sample_weight=None)
print("Error:"+str(preds[0]))
print("Accuracy:"+str(preds[1]))

Epoch 1/60
600/600 [==============================] - 10s 16ms/step - loss: 2.0804 - acc: 0.5817
Epoch 2/60
600/600 [==============================] - 10s 17ms/step - loss: 0.6351 - acc: 0.7817
Epoch 3/60
600/600 [==============================] - 10s 17ms/step - loss: 0.3815 - acc: 0.8483
Epoch 4/60
600/600 [==============================] - 11s 18ms/step - loss: 0.3199 - acc: 0.8850
Epoch 5/60
600/600 [==============================] - 10s 17ms/step - loss: 0.2238 - acc: 0.9200
Epoch 6/60
600/600 [==============================] - 11s 18ms/step - loss: 0.1241 - acc: 0.9600
Epoch 7/60
600/600 [==============================] - 11s 18ms/step - loss: 0.1171 - acc: 0.9633
Epoch 8/60
600/600 [==============================] - 11s 18ms/step - loss: 0.0948 - acc: 0.9650
Epoch 9/60
600/600 [==============================] - 11s 18ms/step - loss: 0.0899 - acc: 0.9717
Epoch 10/60
600/600 [==============================] - 11s 18ms/step - loss: 0.0709 - acc: 0.9817
Epoch 11/60
600/600 [========

In [15]:
# 利用summary函数观察网络结构
HappyModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 70, 70, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 32)        4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 64, 64, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
__________

### 2. 利用Keras搭建Resnet
思路是：（1）组建基本的**残差块**，（2）将残差块组合起来构建Resnet结构
#### 2.1 引入库

In [1]:
import numpy as np
import tensorflow as tf

from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform

import pydot
from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

import resnets_utils 


D:\Tools\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Tools\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Tools\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Tools\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:522: FutureWa

#### 2.2 构建恒等块(Identity Block)
恒等块表示shortcut/skip connection的数据size与经过三次卷积的数据size相等，可以直接相加。

In [10]:
def identity_block(X,f,filters,stage,block):
    """
    参数：
        X - 输入tensor，维度为(m,n_h_prev,n_w_prev,n_c_prev)
        f - 主路径第二个conv层filter的size
        filters - 由三个整数组成的list，分别表示三个conv层的filter个数
        stage,block - 命名用到的数字
    返回：
        X - 恒等块的输出
    """
    # 命名
    conv_name_base = "res"+str(stage)+block+"_branch"
    bn_name_base = "bn"+str(stage)+block+"_branch"
    
    # 获取每一层的filter个数
    (F1,F2,F3) = filters
    
    X_shortcut = X
    
    # 主路径第一层卷积
    X = Conv2D(filters=F1,kernel_size=(1,1),strides=(1,1),padding="valid",
               name=conv_name_base+"2a",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+"2a")(X)
    X = Activation('relu')(X)
    
    # 主路径第二层卷积
    X = Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding="same",
              name=conv_name_base+"2b",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+"2b")(X)
    X = Activation('relu')(X)
    
    # 主路径第三层卷积
    X = Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding="valid",
              name=conv_name_base+"2c",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+"2c")(X)
    
    
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [11]:
# 测试identity_block
tf.reset_default_graph()

with tf.Session() as session:
    np.random.seed(1)
    A_prev = tf.placeholder("float",[3,4,4,6])
    X = np.random.randn(3,4,4,6)
    A = identity_block(A_prev,f=2,filters=[2,4,6],stage=1,block="a")
    
    init = tf.global_variables_initializer()
    session.run(init)
    out = session.run([A],feed_dict={A_prev:X,K.learning_phase():0})
    print("out = "+str(out[0][1][1][0]))
    
    session.close()

out = [0.94822985 0.         1.1610144  2.747859   0.         1.36677   ]


#### 2.3 卷积块（convolutional block）
卷积块使用于shortcut路径上的数据size与三层卷积后的数据size不同的情况，这时需要在shortcut上执行一次卷积操作，使size相同。

In [15]:
def convolutional_block(X,f,filters,stage,block,s=2):
    conv_name_base = "res"+str(stage)+block+"_branch"
    bn_name_base = "bn"+str(stage)+block+"_branch"
    
    (F1,F2,F3) = filters
    
    X_shortcut = X
    
    # 主路径第一层卷积
    X = Conv2D(filters=F1,kernel_size=(1,1),strides=(s,s),padding="valid",
              name=conv_name_base+"2a",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+"2a")(X)
    X = Activation('relu')(X)
    
    # 主路径第二层卷积
    X = Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding="same",
              name=conv_name_base+"2b",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+"2b")(X)
    X = Activation('relu')(X)
    
    # 主路径第三层卷积
    X = Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding="valid",
              name=conv_name_base+"2c",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+"2c")(X)
    X = Activation('relu')(X)
    
    X_shortcut = Conv2D(filters=F3,kernel_size=(1,1),strides=(s,s),padding="valid",
                       name=conv_name_base+"1",kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3,name=bn_name_base+"1")(X)
    
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    return X

In [16]:
# 测试convolutional block
tf.reset_default_graph()

with tf.Session() as session:
    np.random.seed(1)
    A_prev = tf.placeholder("float",[3,4,4,6])
    X = np.random.randn(3,4,4,6)
    A = convolutional_block(A_prev,f=2,filters=[2,4,6],stage=1,block="a")
    
    init = tf.global_variables_initializer()
    session.run(init)
    out = session.run([A],feed_dict={A_prev:X,K.learning_phase():0})
    print("out = "+str(out[0][1][1][0]))
    
    session.close()

out = [0.2572376  0.13451828 0.61327505 0.14167278 1.1710308  0.08300225]


#### 2.4 有残差块和恒等块构建Resnet50

In [17]:
def ResNet50(input_shape=(64,64,3),classes=6):
    
    X_input = Input(input_shape)
    
    # zero padding
    X = ZeroPadding2D((3,3))(X_input)
    
    # stage 1
    X = Conv2D(filters=64,kernel_size=(7,7),strides=(2,2),name="conv1",
              kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D(pool_size=(3,3),strides=(2,2))(X)
    
    # stage 2#stage2
    X = convolutional_block(X, f=3, filters=[64,64,256], stage=2, block="a", s=1)
    X = identity_block(X, f=3, filters=[64,64,256], stage=2, block="b")
    X = identity_block(X, f=3, filters=[64,64,256], stage=2, block="c")
    
    #stage3
    X = convolutional_block(X, f=3, filters=[128,128,512], stage=3, block="a", s=2)
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="b")
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="c")
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="d")
    
    #stage4
    X = convolutional_block(X, f=3, filters=[256,256,1024], stage=4, block="a", s=2)
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="b")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="c")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="d")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="e")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="f")
    
    #stage5
    X = convolutional_block(X, f=3, filters=[512,512,2048], stage=5, block="a", s=2)
    X = identity_block(X, f=3, filters=[512,512,2048], stage=5, block="b")
    X = identity_block(X, f=3, filters=[512,512,2048], stage=5, block="c")
    
    #均值池化层
    X = AveragePooling2D(pool_size=(2,2),padding="same")(X)
    
    #输出层
    X = Flatten()(X)
    X = Dense(classes, activation="softmax", name="fc"+str(classes),
              kernel_initializer=glorot_uniform(seed=0))(X)
    
    
    #创建模型
    model = Model(inputs=X_input, outputs=X, name="ResNet50")
    
    return model

In [18]:
# 编译模型
model = ResNet50(input_shape=(64,64,3),classes=6)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# 获取数据
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = resnets_utils.load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = resnets_utils.convert_to_one_hot(Y_train_orig, 6).T
Y_test = resnets_utils.convert_to_one_hot(Y_test_orig, 6).T

print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))


number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [19]:
# 训练模型
model.fit(X_train,Y_train,epochs=2,batch_size=32)
# 预测
preds = model.evaluate(X_test,Y_test)

print("Error = " + str(preds[0]))
print("Accuracy = " + str(preds[1]))

Epoch 1/2
1080/1080 [==============================] - 216s 200ms/step - loss: 2.8581 - acc: 0.2528
Epoch 2/2
120/120 [==============================] - 10s 81ms/step
Error = 11.039937019348145
Accuracy = 0.17500000049670536
